## Loading Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
import evaluate

In [ ]:
device = torch.device('cuda')

In [ ]:
x = torch.randn(30)
y = torch.randn(30)

z = torch.cat([x.reshape(-1,1), y.reshape(-1,1)], 1)
print(x.shape, y.shape, z.shape)

## Loading Dataset

In [ ]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [ ]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [ ]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [ ]:
train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [0.7, 0.25, 0.05])

train_dataset = Dataset.from_dict(train_dataset[:])
validation_dataset = Dataset.from_dict(validation_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [ ]:
# renaming_dict = {"text_content": "text", "status": "label"}
renaming_dict = {"status": "labels"}

train_dataset = train_dataset.rename_columns(renaming_dict)
validation_dataset = validation_dataset.rename_columns(renaming_dict)
test_dataset = test_dataset.rename_columns(renaming_dict)

### Tokenizing Data

In [ ]:
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents

# We create our normalizer which will appy Unicode normalization and strip accents
normalizer = normalizers.Sequence([NFD(), StripAccents()])

normalizer.normalize_str("Héllò? What aré yòü üptò tòday?")

from tokenizers.pre_tokenizers import Whitespace

# We create our pre-tokenizer which will split based on the regex \w+|[^\w\s]+
pre_tokenizer = Whitespace()

In [ ]:
from tokenizers.models import WordPiece
from tokenizers import Tokenizer

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = normalizer
tokenizer.pre_tokenizer = pre_tokenizer

In [ ]:
from tokenizers.trainers import WordPieceTrainer
import time

# We will create a batch iterator which will generate a batch of sentences for training
# our tokenizaer. This is the preferred way instead of passing single sentences to the
# tokenizer as it will a lot faster.
def batch_iterator(dataset, batch_size=10000):
  for i in range(0, len(dataset), batch_size):
    lower_idx = i
    # Ensure the upper idx doesn't overflow leading to an 'IndexError'
    upper_idx = i + batch_size if i + batch_size <= len(dataset) else len(dataset)
    text = dataset[lower_idx : upper_idx]["title"] + ' ' + dataset[lower_idx : upper_idx]["text_content"]
    yield text
        
# We pass in the list of special tokens so that our model knows about them.
trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

tic = time.perf_counter()
# Now, we do batch training based on our iterator that we defined earlier.
tokenizer.train_from_iterator(batch_iterator(full_dataset), trainer=trainer, length=len(full_dataset))
toc = time.perf_counter()
print(f"Elapsed time: {toc - tic:0.4f} seconds")

In [ ]:
label2id
[label2id[label] for label in train_dataset['labels'][:10]]

In [ ]:
CONTENT_MAX_TEXT_CONTENT = 128
TITLE_MAX_TEXT_CONTENT = 32

def tokenize_func(batch):
  tokenized_batch = dict()
  
  tokenized_batch['labels'] = [label2id[label] for label in batch['labels']]
  
  tokenizer.enable_truncation(max_length=CONTENT_MAX_TEXT_CONTENT)
  tokenizer.enable_padding(length=CONTENT_MAX_TEXT_CONTENT)
  content_out = tokenizer.encode_batch(batch['text_content'])
  
  tokenizer.enable_truncation(max_length=TITLE_MAX_TEXT_CONTENT)
  tokenizer.enable_padding(length=TITLE_MAX_TEXT_CONTENT)
  title_out = tokenizer.encode_batch(batch['title'])
  
  tokenized_batch['content_input_ids'] = torch.IntTensor(list(map(lambda x: x.ids, content_out)))
  tokenized_batch['title_input_ids'] = torch.IntTensor(list(map(lambda x: x.ids, title_out)))
  # print(title)
  
  return tokenized_batch

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

In [ ]:
tokenized_train_dataset

In [ ]:
columns_to_remove = [
  # 'tags_onehot',
  # 'unrecognized_tags_count',
  # 'reputation',
  # 'undeleted_answers',
  # 'user_life_days',
  'title',
  'text_content',
]

tokenized_train_dataset = tokenized_train_dataset.remove_columns(columns_to_remove)
tokenized_validation_dataset = tokenized_validation_dataset.remove_columns(columns_to_remove)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(columns_to_remove)

#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

## Model - Custom

In [ ]:
from custom_model import AutoCompositeModel, TagsModel, NumericalPartModel, TextualPartModel

model_tags = TagsModel(device).to(device)

model_numerical = NumericalPartModel(device).to(device)

model_textual = TextualPartModel(device).to(device)

## Training Setup

### Data Loaders

In [ ]:
def collate_func(batch):
  fixed_batch = pd.DataFrame(batch).to_dict(orient="list")
  # print(fixed_batch)

  return fixed_batch

In [ ]:
from torch.utils.data import DataLoader
training_loader = DataLoader(tokenized_train_dataset, batch_size=128, shuffle=True, collate_fn=collate_func)
validation_loader = DataLoader(tokenized_validation_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(tokenized_test_dataset, batch_size=128, shuffle=True)

In [ ]:
print(tokenized_train_dataset['title_input_ids'][0])
print(len(tokenized_train_dataset['title_input_ids'][0]))

### Loss Function

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# tokenized_train_dataset[0]

### Optimizer

In [ ]:
# Optimizers specified in the torch.optim package
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-5, momentum=0.8)


optimizer_tags = torch.optim.SGD(model_tags.parameters(), lr=1e-3, weight_decay=1e-5)
optimizer_numerical = torch.optim.SGD(model_numerical.parameters(), lr=1e-5, weight_decay=1e-5)
optimizer_textual = torch.optim.SGD(model_textual.parameters(), lr=1e-3, weight_decay=1e-5)

In [ ]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=5).to(device)

### Trainer Setup

In [ ]:
from training_own import Trainer, TrainerConfiguration, get_model_params

In [ ]:
config_tags = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer_tags,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)
trainer_tags = Trainer(model=model_tags, trainer_configuration=config_tags, input_columns=['tags_onehot'], output_column='labels')

In [ ]:
config_numerical = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer_tags,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)
trainer_numerical = Trainer(model=model_numerical, trainer_configuration=config_numerical, input_columns=['unrecognized_tags_count', 'reputation', 'undeleted_answers', 'user_life_days'], output_column='labels')

In [ ]:
config_textual = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer_textual,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)
trainer_textual = Trainer(model=model_textual, trainer_configuration=config_textual, input_columns=['content_input_ids', 'title_input_ids'], output_column='labels')

In [ ]:
get_model_params(model_tags)

In [ ]:
# trainer_tags.train_many_epochs(epochs=2, logging_frequency=100, evaluate_when_logging=False)

In [ ]:
trainer_numerical.train_many_epochs(epochs=4, logging_frequency=100, evaluate_when_logging=False)

In [ ]:
# trainer_textual.train_many_epochs(epochs=2, logging_frequency=100, evaluate_when_logging=False)

In [ ]:

# for i, data in enumerate(training_loader):
#   if i > 0:
#     break
    
#   subset = dict((k, data[k]) for k in ('content_input_ids', 'title_input_ids', 'reputation', 'undeleted_answers', 'user_life_days'))
#   print(subset)
  # text_content = torch.tensor(data['content_input_ids'])
  # print(text_content[0])
  # print(type(text_content[0]))
  
  

In [ ]:
# validation_dataset['tags_onehot']

In [ ]:
# trainer.train_many_epochs(epochs=8, logging_frequency=100, evaluate_when_logging=False)